In [1]:
import time
import numpy as np
import matplotlib.pyplot as plt
import pandas as pd
import random

## Funkcje pomocnicze

In [2]:
# Zmienia format daty z sekund na rrrr-mm-dd
def dataformat(data):
    new_data = time.localtime(data)
    if new_data.tm_mday<10:
        data_day = '0'+str(new_data.tm_mday)
    else:
        data_day = str(new_data.tm_mday)
        
    if new_data.tm_mon<10:
        data_mon = '0' + str(new_data.tm_mon)
    else:
        data_mon =  str(new_data.tm_mon)

    g_data = str(new_data.tm_year) + '-' +data_mon + '-' + data_day
    
    return(g_data)
    

In [3]:
def add_salary(zatr, zwo, pensja):
    zatr_time = time.localtime(zatr)
    #10 sa wyplaty
    if zatr_time.tm_mday<10:
        fir_mon = zatr_time.tm_mon
    else:
        fir_mon = zatr_time.tm_mon+1
    #pierwsza wyplata
    zatr_sec = time.mktime((zatr_time.tm_year, fir_mon, 10, 17, 3, 38, 1, 48, 0))
    pierwsza_pensja = np.round(((zatr_sec - zatr)/(3600*24))*(pensja/30),2)
    zwo_time = time.localtime(zwo)
    if zwo_time.tm_mday<10:
        las_mon = zwo_time.tm_mon
    else:
        las_mon = zwo_time.tm_mon+1
    #ostatnia wyplata
    zwo_sec = time.mktime((zwo_time.tm_year, las_mon, 10, 17, 3, 38, 1, 48, 0))
    ostatnia_pensja = abs(np.round((zwo_sec - zwo)/(3600*24)*(1-(pensja/30)),2))
    wyplata = zatr_sec+31*24*3600
    pensje =[]
    daty = []
    daty.append(dataformat(zatr_sec))
    pensje.append(pierwsza_pensja)
    while wyplata < zwo:
        daty.append(dataformat(wyplata))
        pensje.append(pensja)
        wyplata += 30.5*24*3600
    daty.append(dataformat(zwo_sec))
    pensje.append(ostatnia_pensja)
    return(daty, pensje)

In [4]:
# generuje id, first-pierwsza wartosc, second-druga, i -koncowka
def id_generator(i, first = 0, second= 0):
    if i<10:
        n_id = str(first)+str(second)+'00'+str(i)
    elif i<100 and i >=10 :
        n_id = str(first)+str(second)+'0'+str(i)
    elif i<1000 and i>=100:
        n_id = str(first)+str(second)+str(i)
    return(n_id)

## Klasa zwierzę

In [5]:
class Animal:
    def __init__(self, species):
        
        self.species = species
        
        if self.species == 'pies':
            self.years = 15
            #sample from normal distribution with mean 6 and variance 3 - age of species
            self.mi = 4
            self.sigma = 3
            
        elif self.species == 'kot':
            self.years = 16
            self.mi = 2
            self.sigma = 4
            
        elif self.species == 'szczur':
            self.years = 2.5
            self.mi = 1.25
            self.sigma = 0.5
        self.wizyty = []
        self.daty_wizyt = []
        self.ceny = []
    def stan_zdrowia(self):
        ost_data_wiz = self.daty_wizyt[-1]
        if ost_data_wiz+7*24*3600 < self.actual_time:
            rand = np.random.random()
            if rand > 0.1:
                stan_zdr = "zdrowy"
            else:
                stan_zdr = "chory"
        else:
            stan_zdr = "w czasie leczenia" 
            
        self.stan_zdr = stan_zdr
        return(self.stan_zdr)
        
    def Wizyty(self, ad_time = None): 
        
        ost_data_wizyty = self.daty_wizyt[-1]
        choroby = ["przeziębienie", "zapalenie uszu","świeżb", "zapalenie spojówek","skaleczenie", "inne"]
        if ad_time is not None:
            data_wiz = np.random.uniform(ost_data_wizyty+24*3600, ad_time-24*3600)
        else:
            data_wiz = np.random.uniform(ost_data_wizyty+24*3600, self.actual_time-24*3600)
        zalec = "lek"
        self.daty_wizyt.append(data_wiz)
        chor = choroby[int(np.random.uniform(0, len(choroby)))]
        self.wizyty.append(chor)
        cena = int(np.round(np.random.uniform(50, 150),0))
        self.ceny.append(cena)
        return(data_wiz, chor, zalec, cena) 
    
    def choroby(self):
        
        return(self.wizyty, self.daty_wizyt, self.ceny)
    
    def Species(self):
        if self.species == "pies":
            self.species_id = 1
        elif self.species == "kot":
            self.species_id = 2
        elif self.species == "szczur":
            self.species_id = 3
            
        return(self.species_id)
    
    def plec(self):
        
        rand = np.random.random()
        if rand > 0.5:
            plec = "samica"
        else:
            plec = "samiec"
            
        self.plec = plec
        return(plec)
            
    def name(self, file):
        
        df = pd.read_csv(file, names = ['samce', 'samice'], encoding='unicode_escape')
        df = df.loc[:503]
        
        if self.plec == "samica":
            df = df['samice']
        elif self.plec == "samiec":
            df = df['samce']
        
        rand = np.round(np.random.uniform(0,len(df)-1),0)
        name = (df.values)[int(rand)]
        self.name = name
        
        return(self.name)
        
    def umaszczenie(self):
        
        list_colours = ['bialy', 'czarny', 'szary', 'rudy','brazowy', 'bialo-czarny', 'biszkoptowy', 'bialo-rudy',
                        'bialo-brązowy','bialo-szary','inne']
        len_colours = len(list_colours)
        random_c = np.round(np.random.uniform(0,len_colours-1), 0)
        umaszczenie = list_colours[int(random_c)]
        
        self.umaszczenie = umaszczenie
        
        return(self.umaszczenie, self.species) 

    
    def birth(self, death_year = 0):
        
        years2sec = 365.25*24*3600
        self.actual_time = time.time() - death_year*years2sec  #actual time in second)
        begin_time = self.actual_time - self.years*years2sec #years- how many years back (in seconds)
        sec_mi = self.mi*years2sec
        sec_std = self.sigma*years2sec
        birth_date = np.random.normal(begin_time+sec_mi, (sec_std), 1)
        
        #looking for dates beetwen begin and actual time
        while begin_time > birth_date or self.actual_time < birth_date:
        
            birth_date  = np.random.normal(begin_time+sec_mi, (sec_std), 1)
            
        #birth date of animal    
        self.birth_date = birth_date
        
        return( self.birth_date , self.actual_time)
        
    def masa(self):
        #sprawdzam date urodzenia bo pies majacy 2 miesiace nie bedzie wazyl 40kg
        act = (time.localtime(self.actual_time))
        bir = (time.localtime(self.birth_date))
        if bir.tm_year == act.tm_year:
            if abs(bir.tm_mon-act.tm_mon) < 6:
                if self.species == 'pies':
                    masa = np.random.normal(5,5)
                    while masa < 0.5:
                        masa = np.random.normal(5,5)
                elif self.species == 'kot':
                    masa = np.random.normal(1,1)
                    while masa < 0.4 or masa > 2:
                        masa = np.random.normal(1,1)
                elif self.species == 'szczur':
                    masa = np.random.normal(0.1,0.2)
                    while masa < 0.1 or masa > 0.45:
                        masa = np.random.normal(0.1,0.2)
                
            else:
                if self.species == 'pies':
                    masa = np.random.normal(12,6)
                    while masa < 0.5 or masa > 30:
                        masa = np.random.normal(12,6)
                elif self.species == 'kot':
                    masa = np.random.normal(1,1.3)
                    while masa < 0.6 or masa > 3:
                        masa = np.random.normal(1,1.3)
                elif self.species == 'szczur':
                    masa = np.random.normal(0.2,0.2)
                    while masa < 0.24 or masa > 0.55:
                        masa = np.random.normal(0.2,0.2)
        else:
            if self.species == 'pies':
                masa = np.random.normal(17,7)
                while masa < 1 or masa > 45:
                    masa = np.random.normal(17,7)
            elif self.species == 'kot':
                masa = np.random.normal(3, 2)
                while masa < 1 or masa > 5:
                    masa = np.random.normal(3, 2)
            elif self.species == 'szczur':
                masa = np.random.normal(0.25,0.2)
                while masa < 0.24 or masa > 0.7:
                    masa = np.random.normal(0.25,0.2)
        self.masa  = np.round(masa, 3)

        return(self.masa)
    
    def acceptance(self, o_years = 3.21): #3.21 years - opening of pound
        
        years2sec = 365.25*24*3600
        open_time = self.actual_time - o_years*years2sec
        
        acc_time = self.actual_time+100
        
        while acc_time > self.actual_time or bool(acc_time < self.birth_date+50*24*3600):
#             print(acc_time > self.actual_time)
#             print(bool(acc_time < self.birth_date))
            acc_time = np.random.uniform(open_time, self.actual_time) 
            
        #print(time.localtime(acc_time))
        #acceptance date    
        self.acc_time = acc_time
        p_wizyta = acc_time+24*3600
        self.daty_wizyt.append(p_wizyta)
        self.wizyty.append("pierwsza wizyta")
        self.ceny.append(50)
        return( self.acc_time )
    
    def adoption(self):
        
        day2sec = 24*3600
        ad_time = np.random.uniform(self.acc_time+7*day2sec, self.actual_time-14*day2sec) 
          
        self.ad_time = ad_time
        #self.adoptujacy()
        
        return( self.ad_time )
    
#print(acc_time > self.actual_time)
#print(bool(acc_time < self.birth_date))
#print(time.localtime(acc_time))
#acceptance date  
    def adoptujacy(self, own_names, own_lastnames, own_streets, own_city):
        own_ind0 = int(np.round(np.random.uniform(0,len(own_names)),0)-1)
        own_ind1 = int(np.round(np.random.uniform(0,len(own_lastnames)),0)-1)
        own_ind2 = int(np.round(np.random.uniform(0,len(own_streets)),0)-1)
        own_name = own_names[own_ind0][0]
        own_lastname = own_lastnames[own_ind1][0]
        own_street = own_streets[own_ind2]+' ' + str(int(np.random.uniform(1,100)))
        own_tel = str(int(np.random.uniform(100,999)))+"-"
        own_tel = own_tel + str(int(np.random.uniform(100,999)))+"-"+str(int(np.random.uniform(100,999)))
        owner = {}
        owner['name'] = own_name
        owner['last_name'] = own_lastname
        owner['adres'] = own_street + ' ' + own_city
        owner['telefon'] = own_tel
        
        return(owner)
            
        
    def kastracja(self):
        rand = np.random.random()
        if rand > 0.1:
            self.kastracja = "tak"
            n_rand = np.random.random()
            if n_rand > 0.5:
                self.k_day = self.acc_time+ np.random.uniform(10,35)*24*3600
                self.daty_wizyt.append(self.k_day)
                self.wizyty.append("sterylizacja")
                self.ceny.append(200)
            else:
                self.k_day = self.birth_date + np.random.uniform(200,350)*24*3600

        else:
            self.kastracja = "nie"
            self.k_day = 0
        
        
        return(self.kastracja, self.k_day)
    def next_acceptance(self):
        day2sec = 24*3600
        new_acc_time = 0
        #20% zwierzakow wraca do schorniska
        if self.ad_time is not None:
            new_acc_time = np.random.uniform(self.ad_time+3*day2sec, self.actual_time-7*day2sec ) 
        #print(acc_time > self.actual_time)
        #print(bool(acc_time < self.birth_date))
        #print(time.localtime(acc_time))
        #acceptance date    
        self.acc_time = new_acc_time
        p_wizyta = self.acc_time+24*3600
        self.daty_wizyt.append(p_wizyta)
        self.wizyty.append("pierwsza wizyta")
        self.ceny.append(50)
        return( self.acc_time )
    
    
#sterylizacja - klasa zwierze - data wizty +1 dzien +tydzien sterylizacja
#0.01 0.01
#2 lata
#aktu -200
# wykladniczy ale od pewnego momentu z paramtrem 1
#masa ciała
#400 - 95% adotptowane , 5% zmarłe
#adopcje
# maks - 5, srednio -2, procent wracajacych - 30% 
# w wiekszosci darowizna, co miesiac -wyplaty, dotacja z urzedu
#imiona - nie powtarzaja sie - usuwam z listy uzyte imie


## Funkcja pracownik

In [6]:
def pracownik(own_names, own_lastnames, own_streets, own_city, years=3.21):
        own_ind0 = int(np.round(np.random.uniform(0,len(own_names)),0)-1)
        own_ind1 = int(np.round(np.random.uniform(0,len(own_lastnames)),0)-1)
        own_ind2 = int(np.round(np.random.uniform(0,len(own_streets)),0)-1)
        own_name = own_names[own_ind0][0]
        own_lastname = own_lastnames[own_ind1][0]
        own_street = own_streets[own_ind2]+' ' + str(int(np.random.uniform(1,100)))
        own_tel = str(int(np.random.uniform(100,999)))+"-"
        own_tel = own_tel + str(int(np.random.uniform(100,999)))+"-"+str(int(np.random.uniform(100,999)))
        owner = {}
        owner['name'] = own_name
        owner['last_name'] = own_lastname
        owner['adres'] = own_street + ' ' + own_city
        owner['telefon'] = own_tel
        years2sec = 365.25*24*3600
        actual_time = time.time()
        open_time = actual_time - years*years2sec
        
        data_zat = np.random.uniform(open_time, actual_time)
        data_zw =  np.random.uniform(data_zat, actual_time+24*3600)
        owner["zatr"] = data_zat
        owner["zwoln"] = data_zw
        return(owner)

In [7]:
ulice = ['ul. Agrestowa',
'ul. Akacjowa',
'ul. Armii Krajowej',
'ul. Balonowa',
'ul. Batalionów Chłopskich',
'ul. gen. Józefa Bema',
'ul. Błękitna',
'ul. Bociania',
'ul. Jerzego Bocka',
'ul. Boczna',
'ul. Bolesława Krzywoustego',
'ul. Arkadiusza Bożka',
'ul. Bratnia',
'ul. Brzozowa',
'ul. Ceglana',
'ul. Fryderyka Chopina',
'ul. Cicha',
'ul. Ciepła',
'ul. Antoniego Cieszyńskiego',
'ul. Cmentarna',
'ul. Bronisława Czecha',
'ul. Ignacego Daszyńskiego',
'ul. Henryka Dąbrowskiego',
'ul. Demokratów',
'ul. Kazimierza Deyny',
'ul. Dobroszycka',
'ul. Energetyczna',
'ul. Fabryczna',
'ul. Gliniana',
'ul. Władysława Grabskiego',
'ul. Gryczana',
'ul. gen. Józefa Hallera',
'ul. św. Jadwigi',
'ul. Jagodowa',
'ul. Jaśminowa',
'ul. Jarzynowa',
'ul. Jasna',
'ul. Jastrzębia',
'ul. Jęczmienna',
'ul. Kasztanowa',
'ul. Kazimierza Wielkiego',
'ul. Jana Kilińskiego',
'ul. gen. Franciszka Kleeberga',
'ul. Klonowa',
'ul. Jana Kochanowskiego',
'ul. Kolejowa',
'ul. Kościelna',
'ul. Tadeusza Kościuszki',
'skwer Kombatantów Rzeczypospolitej',
'ul. Mikołaja Kopernika',
'ul. Kosynierów',
'ul. Krótka',
'ul. Kresowa',
'ul. Krucza',
'ul. Krzywa',
'ul. Janusza Kusocińskiego',
'ul. Eugeniusza Felicjana Kwiatkowskiego',
'ul. Kwiatowa',
'ul. Joachima Lelewela',
'ul. Leśna',
'ul. Leszczynowa',
'ul. Bolesława Limanowskiego',
'ul. Lipowa',
'ul. 11 Listopada',
'ul. Lotnicza',
'ul. Stefana Żeromskiego',
'ul. Żwirki i Wigury',
'ul. Żytnia',
'ul. Ludwikowska',
'ul. Lwowska',
'ul. Henryka Łasaka',
'ul. Łąkowa',
'ul. Łowiecka',
'ul. Łukanowska',
'ul. Łużycka',
'pl. Książąt Śląskich',
'ul. Tadeusza Ślusarskiego',
'ul. Świerkowa',
'ul. gen. Stanisława Maczka',
'ul. 3 Maja',
'ul. Malinowa',
'ul. Bronisława Malinowskiego',
'ul. Małopolna',
'ul. Jana Matejki',
'ul. Adama Mickiewicza',
'ul. Stanisława Mikołajczyka',
'ul. Miła',
'ul. Miodowa',
'ul. Młynarska',
'ul. Mokra',
'ul. Ignacego Mościckiego',
'ul. Stanisława Moniuszki',
'ul. Na Podkowie',
'ul. Gabriela Narutowicza',
'ul. Nowowiejska',
'ul. Ogrodowa',
'ul. Okrężna',
'ul. Orla',
'ul. Osada Bystre',
'ul. Osiedle',
'ul. Otwarta',
'ul. Owocowa',
'ul. Owsiana',
'ul. Ignacego Paderewskiego',
'ul. Parkowa',
'ul. Północna',
'ul. Pocztowa',
'ul. Podchorążych',
'ul. Pogodna',
'ul. Polna',
'ul. Południowa',
'ul. Księcia Józefa Poniatowskiego',
'ul. Poprzeczna',
'ul. Porzeczkowa',
'ul. Powietrzna',
'ul. Poziomkowa',
'ul. Prosta',
'ul. Bolesława Prusa',
'ul. Przemysłowa',
'ul. Kazimierza Przerwy-Tetmajera',
'ul. Przyjaźni',
'ul. Bogusława Psujka',
'ul. Pszeniczna',
'ul. Ptasia',
'ul. Kazimierza Pułaskiego',
'ul. Heleny Radlińskiej',
'ul. Różana',
'ul. Mikołaja Reja',
'ul. Tadeusza Rejtana',
'ul. Władysława Stanisława Reymonta',
'rondo Rondo Jana Pawła II',
'rondo Rondo Miast Partnerskich',
'ul. Wandy Rutkiewicz',
'ul. Rycerska',
'ul. Rzemieślnicza',
'ul. Rzepakowa',
'ul. Sejmowa',
'ul. Janusza Sidły',
'ul. Henryka Sienkiewicza',
'ul. gen. Władysława Sikorskiego',
'ul. Jana Sinapiusa',
'ul. Marii Skłodowskiej-Curie',
'ul. Słoneczna',
'ul. Juliusza Słowackiego',
'ul. Sokola',
'ul. Ignacego Solarza',
'rondo Solidarności',
'ul. Sosnowa',
'ul. Spacerowa',
'ul. Spadochronowa',
'ul. Spokojna',
'ul. Feliksa Stamma',
'ul. Jana Stapińskiego',
'pl. Stanisława Staszica',
'ul. Stawowa',
'ul. Stolarska'
'ul. mjr. Henryka Sucharskiego',
'ul. ks. Franciszka Sudoła',
'ul. Szkolna',
'ul. Szybowcowa',
'ul. Targowa',
'ul. Tęczowa',
'ul. Lwa Tołstoja',
'ul. Trakcyjna',
'ul. Romualda Traugutta',
'ul. Stanisławy Walasiewicz',
'ul. Wałowa',
'ul. Wały Jagiellońskie',
'ul. Wądoły',
'ul. Wąska',
'ul. Wesoła',
'ul. Wiejska',
'ul. Wielkopolna',
'ul. Wierzbowa',
'ul. Wikliniarska',
'ul. Wileńska',
'ul. Wiśniowa',
'ul. Wincentego Witosa',
'ul. Wojska Polskiego',
'ul. Wrocławska',
'ul. Wschodnia',
'ul. Wspólna',
'ul. Józefa Wybickiego',
'ul. Zakładowa',
'ul. Zamkowa',
'ul. Zielona'
]

In [8]:
# zmieniam polskie znaki w nazwach ulic
for i in range(len(ulice)):
    ulice[i] = ulice[i].replace('ą','a')
    ulice[i] = ulice[i].replace('ć','c')
    ulice[i] = ulice[i].replace('Ć','C')
    ulice[i] = ulice[i].replace('ę','e')
    ulice[i] = ulice[i].replace('ł','l')
    ulice[i] = ulice[i].replace('Ł','L')
    ulice[i] = ulice[i].replace('ń','n')
    ulice[i] = ulice[i].replace('Ń','N')
    ulice[i] = ulice[i].replace('ó','o')
    ulice[i] = ulice[i].replace('ś','s')
    ulice[i] = ulice[i].replace('Ś','S')
    ulice[i] = ulice[i].replace('ż','z')
    ulice[i] = ulice[i].replace('ź','z')
    ulice[i] = ulice[i].replace('Ż','Z')
    ulice[i] = ulice[i].replace('Ź','Z')

## Dane

In [9]:
names_male = pd.read_csv('./dane/n_male.csv', names = ['names'], encoding='unicode_escape')
names_female = pd.read_csv('./dane/n_female.csv', names = ['names'], encoding='unicode_escape')
lastnames_male = pd.read_csv('./dane/lastn_male.csv',names = ['names'], encoding='unicode_escape')
lastnames_female = pd.read_csv('./dane/lastn_female.csv',names = ['names'], encoding='unicode_escape')
miasta = ["Wroclaw", "Olesnica", "Opole"]
names_male = (names_male.values).tolist() 
lastnames_male = (lastnames_male.values).tolist()
names_female = (names_female.values).tolist() 
lastnames_female = (lastnames_female.values).tolist()

## Schronisko

In [10]:
informacje = []
gatunki = []
zwierz_schr = []
zwierz_byle = []
adoptujace = []
hist_zdr = []
transakcje = []

In [11]:
actual_time = time.time()
#tutaj zbieram dane o pracownikach
pr_id = []
o_years = 3.21 #lata działania schroniska

years2sec = 365.25*24*3600
#data otwarcia schroniska
open_time = actual_time - o_years*years2sec
zwolnienia  = []

## Pracownicy

### Byli opiekunowie

In [12]:
byli_opiekunowie = [] 
opiekunowie = []
for i in range(4):
    
    p_id = id_generator(i=i, first=2)
    rand = np.random.random()
    if rand>0.5:
        opiekun = pracownik(names_male, lastnames_male, ulice, "Wroclaw")
    else:
        opiekun = pracownik(names_female, lastnames_female, ulice, "Wroclaw")
    
    
    zwol = opiekun["zwoln"]
    pensja = 2500
    data, pensja = add_salary(open_time,zwol,pensja)
    for j in range(len(data)):
        trans = {}
        trans['data'] = data[j]
        trans['id_transakcji'] = id_generator(i=int(str(i)+str(j)), first=5, second = 9)
        trans['imie'] = opiekun['name']
        trans['nazwisko'] = opiekun['last_name']
        trans['kwota'] = 0-float(pensja[j])
        trans['tytul'] = 'wyplata'
        transakcje.append(trans)
    zatr = dataformat(open_time)
    zwolnienia.append(zwol)
    zwo = dataformat(zwol)
    opiekun_d = {'id':p_id, 'imie': opiekun['name'], 'nazwisko': opiekun['last_name'], 
               'adres': opiekun['adres'], 'telefon': opiekun['telefon'], 'zatrudnienie':zatr, "zwolenienie": zwo,
          'stanowisko': 'opiekun', 'pensja': 2500}
    byli_opiekunowie.append(opiekun_d)
    opiekunowie.append(opiekun_d)
    
    
pd.DataFrame(data=byli_opiekunowie)


,adres,id,imie,nazwisko,pensja,stanowisko,telefon,zatrudnienie,zwolenienie
0,ul. Szybowcowa 18 Wroclaw,20000,MICHAL,WALKOWSKI,2500,opiekun,510-602-795,2016-04-04,2019-03-16
1,ul. Stanislawa Mikolajczyka 82 Wroclaw,20001,JAKUB,DEBICKI,2500,opiekun,705-264-598,2016-04-04,2019-05-05
2,ul. Lakowa 1 Wroclaw,20002,IRENA,BOCZEK,2500,opiekun,933-121-535,2016-04-04,2017-07-26
3,ul. Ignacego Solarza 58 Wroclaw,20003,KAZIMIERZ,RATAJCZAK,2500,opiekun,639-141-680,2016-04-04,2019-03-08


### Aktualni opiekunowie

In [13]:
#najpierw opiekunowie
#4 opiekunow pracuje aktualnie
for i in range(4):

    rand = np.random.random()
    if rand>0.5:
        opiekun = pracownik(names_male, lastnames_male, ulice, "Wrocław")
    else:
        opiekun = pracownik(names_female, lastnames_female, ulice, "Wrocław")
    zat = zwolnienia[i]+np.random.uniform(1,30)*24*3600
    zatr = dataformat(zat)
    pensja = 2500
    data, pensja = add_salary(zat,actual_time,pensja)
    for j in range(len(data)):
        trans = {}
        trans['data'] = data[j]
        trans['id_transakcji'] = id_generator(i=int(str(i)+str(j)), first=5, second = 9)
        trans['imie'] = opiekun['name']
        trans['nazwisko'] = opiekun['last_name']
        trans['kwota'] = 0-float(pensja[j])
        trans['tytul'] = 'wyplata'
        transakcje.append(trans)
    zwo = "9999-99-99"
    p_id = id_generator(i=i, first=2)
    pr_id.append(p_id)
    opiekun_d = {'id':p_id, 'imie': opiekun['name'], 'nazwisko': opiekun['last_name'], 
               'adres': opiekun['adres'], 'telefon': opiekun['telefon'], 'zatrudnienie':zatr, "zwolenienie": zwo,
          'stanowisko': 'opiekun', 'pensja': 2500}
    opiekunowie.append(opiekun_d)


### Reszta pracowników

In [14]:
#kierowca, ksiegowy, kierownik, asystent kierownika -pracuja od poczatku
pensje = ['3000','3500','5000', '4000']
stanowiska = ['kierowca', 'księgowy', 'kierownik', 'asystent kierownika']
for i in range(4):
    p_id = id_generator(i=i, first=1)
    rand = np.random.random()
    if rand>0.5:
        opiekun = pracownik(names_male, lastnames_male, ulice, "Wroclaw")
    else:
        opiekun = pracownik(names_female, lastnames_female, ulice, "Wroclaw")
    
    data, pensja = add_salary(open_time,actual_time,eval(pensje[i]))
    for j in range(len(data)):
        trans = {}
        trans['data'] = data[j]
        trans['id_transakcji'] = id_generator(i=int(str(i)+str(j)), first=5, second = 9)
        trans['imie'] = opiekun['name']
        trans['nazwisko'] = opiekun['last_name']
        trans['kwota'] = 0-float(pensja[j])
        trans['tytul'] = 'wyplata'
        transakcje.append(trans)
    zatr = dataformat(open_time)
    zwo = "9999-99-99"


    opiekun_d = {'id':p_id, 'imie': opiekun['name'], 'nazwisko': opiekun['last_name'], 
               'adres': opiekun['adres'], 'telefon': opiekun['telefon'], 'zatrudnienie':zatr, "zwolenienie": zwo,
          'stanowisko': stanowiska[i], 'pensja': pensje[i]}
    opiekunowie.append(opiekun_d)


In [15]:
pd.DataFrame(data=opiekunowie)

,adres,id,imie,nazwisko,pensja,stanowisko,telefon,zatrudnienie,zwolenienie
0,ul. Szybowcowa 18 Wroclaw,20000,MICHAL,WALKOWSKI,2500,opiekun,510-602-795,2016-04-04,2019-03-16
1,ul. Stanislawa Mikolajczyka 82 Wroclaw,20001,JAKUB,DEBICKI,2500,opiekun,705-264-598,2016-04-04,2019-05-05
2,ul. Lakowa 1 Wroclaw,20002,IRENA,BOCZEK,2500,opiekun,933-121-535,2016-04-04,2017-07-26
3,ul. Ignacego Solarza 58 Wroclaw,20003,KAZIMIERZ,RATAJCZAK,2500,opiekun,639-141-680,2016-04-04,2019-03-08
4,ul. Mokra 66 Wrocław,20000,AGNIESZKA,TOMCZAK,2500,opiekun,506-837-134,2019-03-18,9999-99-99
5,ul. Janusza Sidly 89 Wrocław,20001,ZUZANNA,KANIA,2500,opiekun,400-497-263,2019-05-27,9999-99-99
6,ul. Demokratow 65 Wrocław,20002,MONIKA,MODLINSKA,2500,opiekun,654-276-139,2017-08-02,9999-99-99
7,ul. Jastrzebia 41 Wrocław,20003,ALICJA,MIKOSZ,2500,opiekun,429-486-138,2019-04-02,9999-99-99
8,ul. gen. Wladyslawa Sikorskiego 64 Wroclaw,10000,SZYMON,FILIPIUK,3000,kierowca,779-215-278,2016-04-04,9999-99-99
9,ul. Janusza Kusocinskiego 77 Wroclaw,10001,AGATA,LESZCZYK,3500,księgowy,312-918-330,2016-04-04,9999-99-99


### Wolontariusze

In [16]:
#przez schronisko przewinelo sie 67 wolontariuszy
wolontariusze = []
for i in range(67):
    p_id = id_generator(i=i, first=4)
        
    rand = np.random.random()
    if rand>0.5:
        opiekun = pracownik(names_male, lastnames_male, ulice, "Wroclaw")
    else:
        opiekun = pracownik(names_female, lastnames_female, ulice, "Wroclaw")
        
    zatr = dataformat(opiekun['zatr'])
    
    
    zwol = opiekun["zwoln"]
    zwo = dataformat(zwol)
    
    if np.random.random() > 0.9:
        zwo = '9999-99-99'
    
    wolontariusz =  {'id':p_id, 'imie': opiekun['name'], 'nazwisko': opiekun['last_name'], 
               'adres': opiekun['adres'], 'telefon': opiekun['telefon'], 'zatrudnienie':zatr, 
                     "rezygnacja": zwo}
    wolontariusze.append(wolontariusz)


In [17]:
pd.DataFrame(data=wolontariusze)

,adres,id,imie,nazwisko,rezygnacja,telefon,zatrudnienie
0,ul. Lwowska 74 Wroclaw,40000,DARIUSZ,STALMACH,2019-05-07,968-119-968,2018-10-26
1,ul. Miodowa 42 Wroclaw,40001,BARTOSZ,BARAN,2019-06-04,362-951-153,2018-06-21
2,ul. Energetyczna 6 Wroclaw,40002,DARIUSZ,GLOWACZ,2019-03-24,931-986-605,2016-11-17
3,ul. Osiedle 16 Wroclaw,40003,RAFAL,MENDYK,2017-10-01,458-369-485,2016-09-21
4,ul. Wandy Rutkiewicz 94 Wroclaw,40004,FRANCISZEK,KRUCZKOWSKI,2019-04-15,209-165-902,2017-09-05
5,ul. Sejmowa 66 Wroclaw,40005,JERZY,KLEPACZ,2018-12-06,420-335-136,2018-09-09
6,ul. Stefana Zeromskiego 29 Wroclaw,40006,ADAM,MODRZYNSKI,2017-05-11,994-546-370,2016-06-23
7,ul. Orla 87 Wroclaw,40007,ANNA,GRZESKOWIAK,2018-02-19,213-922-378,2017-04-29
8,ul. Ptasia 50 Wroclaw,40008,WIKTORIA,SLOMINSKA,2019-04-09,507-164-421,2019-01-03
9,ul. Kasztanowa 86 Wroclaw,40009,BARTOSZ,ZDUNCZYK,2017-04-18,352-784-387,2016-06-25


### Weterynarze

In [18]:
# 3 weterynarzy
#najpierw opiekunowie
#4 opiekunow pracuje aktualnie
weterynarze =[] 
w_id = []
miastaW = ["Wroclaw", "Wroclaw", "Olesnica","Wroclaw"]
for i in range(4):
    rand = np.random.random()
    if rand>0.5:
        opiekun = pracownik(names_male, lastnames_male, ulice, miastaW[i])
    else:
        opiekun = pracownik(names_female, lastnames_female, ulice, miastaW[i])

    zatr = dataformat(open_time)
    zwo = "9999-99-99"
    p_id = id_generator(i=i, first=3)
    w_id.append(p_id)
    opiekun_d = {'id':p_id, 'imie': opiekun['name'], 'nazwisko': opiekun['last_name'], 
               'adres': opiekun['adres'], 'telefon': opiekun['telefon'], 'zatrudnienie':zatr, "zwolenienie": zwo,
          'stanowisko': 'weterynarz'}
    weterynarze.append(opiekun_d)


In [19]:
pd.DataFrame(data=weterynarze)

,adres,id,imie,nazwisko,stanowisko,telefon,zatrudnienie,zwolenienie
0,ul. Jarzynowa 77 Wroclaw,30000,DOROTA,STEPINSKA,weterynarz,767-173-223,2016-04-04,9999-99-99
1,ul. Targowa 59 Wroclaw,30001,JAN,IWANCZUK,weterynarz,423-800-310,2016-04-04,9999-99-99
2,ul. Demokratow 95 Olesnica,30002,ALICJA,PROKOP,weterynarz,750-474-297,2016-04-04,9999-99-99
3,ul. Stolarskaul. mjr. Henryka Sucharskiego 7 W...,30003,LESZEK,PIELA,weterynarz,188-219-944,2016-04-04,9999-99-99


# Zwierzęta

### gatunki

In [20]:
#gatunki
gatunki = {'nazwa' : ['pies', 'kot', 'szczur'], 'id_gatunku': [1,2,3]}
gat = pd.DataFrame(data=gatunki)
gat

,nazwa,id_gatunku
0,pies,1
1,kot,2
2,szczur,3


## W schronisku

### pieski

In [21]:
#92 psy- tyle liczy aktualnie nasze schronisko, 41 kotow, i 17 szczurow
len_dog = 92
k = 1
l = 1
for i in range(len_dog):
    dog_info = {}
    dog_schr = {}
    dog_wiz = {}
    dog_trans = {}
    dog = Animal(species = 'pies')
    birth, _ = dog.birth()
    birth_d = dataformat(birth)
    acc = dog.acceptance()
    if np.random.random()>0.7:
        dog_info['wcześniej adoptowany'] = "tak"
        adop = dog.adoption()
        #ponowne przyjecie
        acc = dog.next_acceptance()
        if np.random.random()>0.9:
            adop = dog.adoption()
            #ponowne przyjecie
            acc = dog.next_acceptance()
    else:
        dog_info['wcześniej adoptowany'] = "nie"

    dog_info['data_urodzenia'] = birth_d
    dog_info['data_przyjecia'] = dataformat(acc)
    
    dog_info['plec'] = dog.plec()
    dog_info['masa'] = dog.masa()
    dog_info['imie'] = dog.name('./dane/npieski.csv')
    kastracja = dog.kastracja()
    dog_info['id_gat'] = 1
    dog_schr['id_gat'] = dog_info['id_gat']

    dog_info['id'] = id_generator(i=i, first=6, second =1)
        
    dog_schr['id'] = dog_info['id']
        
    if np.random.random()>0.9:
        dog_info['inne_zwierzeta'] = "agresywny"
    else:
        dog_info['inne_zwierzeta'] = "łagodny"
    umaszczenie= dog.umaszczenie()
    
    dog_info['umaszczenie']=umaszczenie[0]
    dog_info['gatunek'] = umaszczenie[1]
    
    if dog_info['masa']>=20:
        dog_info['ID opiekuna'] = [pr_id[0]]
        dog_info['blok'] = 'A_'+str(k)
        k += 1
    else:
        dog_info['ID opiekuna'] = [pr_id[1]]
        dog_info['blok'] = 'B_'+str(l)
        l += 1
    dog_schr['ID opiekuna'] = dog_info['ID opiekuna']
    dog_schr['blok'] = dog_info['blok']
    ## czy nadaje sie do adopcji?
    if np.random.random()>0.1:
        data_wiz, chor, zalec, cena = dog.Wizyty()
        if np.random.random()>0.7:
            data_wiz1, chor1, zalec1, cena1 = dog.Wizyty()
    zdrowie = dog.stan_zdrowia()
    if zdrowie  == "zdrowy":
        dog_schr['Stan'] = "nadaje się do adopcji"
    else:
        dog_schr['Stan'] = "nie nadaje się do adopcji - "+str(zdrowie)
    wizyty, daty, ceny = dog.choroby()
    for j in range(len(wizyty)):
        dog_wiz['ID_wizyty'] =  id_generator(i = i, first = 4, second = j)
        dog_wiz['id'] = dog_info['id']
        index =int(np.round(np.random.uniform(0,3.49),0))
        id_pr =  w_id[index]
        dog_wiz['ID pracownika'] = id_pr
        dog_wiz['data wizyty'] = dataformat(daty[j])
        dog_wiz['diagnoza'] = wizyty[j]
        dog_wiz['zalecenia'] = 'lek'
        dog_wiz['stan zdrowia'] = zdrowie
        dog_wiz['sterylizacja'] = kastracja[0]
        dog_trans['id_transakcji'] = id_generator(i = i, first = 5, second = j)
        dog_trans['tytul']='weterynarz, id_zwierzecia:'+ str(dog_info['id'])
        dog_trans['kwota'] = 0-ceny[j]
        dog_trans['data'] = dataformat(daty[j])
        dog_trans['imie'] = weterynarze[index]['imie']
        dog_trans['nazwisko'] = weterynarze[index]['nazwisko']
    transakcje.append(dog_trans)
    hist_zdr.append(dog_wiz)
    zwierz_schr.append(dog_schr)
    informacje.append(dog_info)
    
    

### kotki

In [22]:
#92 psy- tyle liczy aktualnie nasze schronisko. 
len_cat = 41
k = 0
for i in range(len_cat):
    dog_info = {}
    dog_schr = {}
    dog_wiz = {}
    dog_trans = {}
    dog = Animal(species = 'kot')
    birth, _ = dog.birth()
    birth_d = dataformat(birth)
    acc = dog.acceptance()
    if np.random.random()>0.7:
        dog_info['wcześniej adoptowany'] = "tak"
        adop = dog.adoption()
        #ponowne przyjecie
        acc = dog.next_acceptance()
        if np.random.random()>0.9:
            adop = dog.adoption()
            #ponowne przyjecie
            acc = dog.next_acceptance()
    else:
        dog_info['wcześniej adoptowany'] = "nie"

    dog_info['data_urodzenia'] = birth_d
    dog_info['data_przyjecia'] = dataformat(acc)
    
    dog_info['plec'] = dog.plec()
    dog_info['masa'] = dog.masa()
    dog_info['imie'] = dog.name('./dane/npieski.csv')
    kastracja = dog.kastracja()
    dog_info['id_gat'] = 2
    dog_schr['id_gat'] = dog_info['id_gat']
    dog_info['id'] = id_generator(i=i, first=6, second =2)
        
        
    dog_schr['id'] = dog_info['id']
        
    if np.random.random()>0.9:
        dog_info['inne_zwierzeta'] = "agresywny"
    else:
        dog_info['inne_zwierzeta'] = "łagodny"
    umaszczenie= dog.umaszczenie()
    
    dog_info['umaszczenie']=umaszczenie[0]
    dog_info['gatunek'] = umaszczenie[1]
    
    dog_info['ID opiekuna'] = [pr_id[3]]
    if i%3==0:
        dog_info['blok'] = 'C_'+str(i//3)
        k = i
    else:
        dog_info['blok'] = 'C_'+str(k//3)
    
    dog_schr['ID opiekuna'] = dog_info['ID opiekuna']
    dog_schr['blok'] = dog_info['blok']
    ## czy nadaje sie do adopcji?
    if np.random.random()>0.1:
        data_wiz, chor, zalec, cena = dog.Wizyty()
        if np.random.random()>0.7:
            data_wiz1, chor1, zalec1, cena1 = dog.Wizyty()
    zdrowie = dog.stan_zdrowia()
    if zdrowie  == "zdrowy":
        dog_schr['Stan'] = "nadaje się do adopcji"
    else:
        dog_schr['Stan'] = "nie nadaje się do adopcji - "+str(zdrowie)
    wizyty, daty, ceny = dog.choroby()
    for j in range(len(wizyty)):
        dog_wiz['ID_wizyty'] =  id_generator(i = i, first = 4, second = j)
        dog_wiz['id'] = dog_info['id']
        index =int(np.round(np.random.uniform(0,3.49),0))
        id_pr =  w_id[index]
        dog_wiz['ID pracownika'] = id_pr
        dog_wiz['data wizyty'] = dataformat(daty[j])
        dog_wiz['diagnoza'] = wizyty[j]
        dog_wiz['zalecenia'] = 'lek'
        dog_wiz['stan zdrowia'] = zdrowie
        dog_wiz['sterylizacja'] = kastracja[0]
        dog_trans['id_transakcji'] = id_generator(i = i, first = 5, second = j)
        dog_trans['tytul']='weterynarz, id_zwierzecia:'+ str(dog_info['id'])
        dog_trans['kwota'] = 0 - ceny[j]
        dog_trans['data'] = dataformat(daty[j])
        dog_trans['imie'] = weterynarze[index]['imie']
        dog_trans['nazwisko'] = weterynarze[index]['nazwisko']
    transakcje.append(dog_trans)
    hist_zdr.append(dog_wiz)
    zwierz_schr.append(dog_schr)
    informacje.append(dog_info)
    
    

### szczurki

In [23]:
#92 psy- tyle liczy aktualnie nasze schronisko. 
len_rat = 17
k=0
for i in range(len_rat):
    dog_info = {}
    dog_schr = {}
    dog_wiz = {}
    dog_trans = {}
    dog = Animal(species = 'szczur')
    birth, _ = dog.birth()
    birth_d = dataformat(birth)
    acc = dog.acceptance()
    if np.random.random()>0.7:
        dog_info['wcześniej adoptowany'] = "tak"
        adop = dog.adoption()
        #ponowne przyjecie
        acc = dog.next_acceptance()
        if np.random.random()>0.9:
            adop = dog.adoption()
            #ponowne przyjecie
            acc = dog.next_acceptance()
    else:
        dog_info['wcześniej adoptowany'] = "nie"

    dog_info['data_urodzenia'] = birth_d
    dog_info['data_przyjecia'] = dataformat(acc)
    
    dog_info['plec'] = dog.plec()
    dog_info['masa'] = dog.masa()
    dog_info['imie'] = dog.name('./dane/npieski.csv')
    kastracja = dog.kastracja()
    dog_info['id_gat'] = 2
    dog_schr['id_gat'] = dog_info['id_gat']
    
    dog_info['id'] = id_generator(i=i, first=6, second =3)
        
        
    dog_schr['id'] = dog_info['id']
        
    if np.random.random()>0.9:
        dog_info['inne_zwierzeta'] = "agresywny"
    else:
        dog_info['inne_zwierzeta'] = "łagodny"
    umaszczenie= dog.umaszczenie()
    
    dog_info['umaszczenie']=umaszczenie[0]
    dog_info['gatunek'] = umaszczenie[1]
    
    dog_info['ID opiekuna'] = [pr_id[3]]
    dog_info['blok'] = 'C_14'
    
    dog_schr['ID opiekuna'] = dog_info['ID opiekuna']
    dog_schr['blok'] = dog_info['blok']
    ## czy nadaje sie do adopcji?
    if np.random.random()>0.1:
        data_wiz, chor, zalec, cena = dog.Wizyty()
        if np.random.random()>0.7:
            data_wiz1, chor1, zalec1, cena1 = dog.Wizyty()
    zdrowie = dog.stan_zdrowia()
    if zdrowie  == "zdrowy":
        dog_schr['Stan'] = "nadaje się do adopcji"
    else:
        dog_schr['Stan'] = "nie nadaje się do adopcji - "+str(zdrowie)
    wizyty, daty, ceny = dog.choroby()
    for j in range(len(wizyty)):
        dog_wiz['ID_wizyty'] =  id_generator(i = i, first = 4, second = j)
        dog_wiz['id'] = dog_info['id']
        index =int(np.round(np.random.uniform(0,3.49),0))
        id_pr =  w_id[index]
        dog_wiz['ID pracownika'] = id_pr
        dog_wiz['data wizyty'] = dataformat(daty[j])
        dog_wiz['diagnoza'] = wizyty[j]
        dog_wiz['zalecenia'] = 'lek'
        dog_wiz['stan zdrowia'] = zdrowie
        dog_wiz['sterylizacja'] = kastracja[0]
        dog_trans['id_transakcji'] = id_generator(i = i, first = 5, second = j)
        dog_trans['tytul']='weterynarz, id_zwierzecia:'+ str(dog_info['id'])
        dog_trans['kwota'] = 0- ceny[j]
        dog_trans['data'] = dataformat(daty[j])
        dog_trans['imie'] = weterynarze[index]['imie']
        dog_trans['nazwisko'] = weterynarze[index]['nazwisko']
    transakcje.append(dog_trans)
    hist_zdr.append(dog_wiz)
    zwierz_schr.append(dog_schr)
    informacje.append(dog_info)
    
    

In [24]:
pd.DataFrame(transakcje)

,data,id_transakcji,imie,kwota,nazwisko,tytul
0,2016-04-10,59000,MICHAL,-532.83,WALKOWSKI,wyplata
1,2016-05-11,59001,MICHAL,-2500.00,WALKOWSKI,wyplata
2,2016-06-11,59002,MICHAL,-2500.00,WALKOWSKI,wyplata
3,2016-07-11,59003,MICHAL,-2500.00,WALKOWSKI,wyplata
4,2016-08-11,59004,MICHAL,-2500.00,WALKOWSKI,wyplata
5,2016-09-10,59005,MICHAL,-2500.00,WALKOWSKI,wyplata
6,2016-10-11,59006,MICHAL,-2500.00,WALKOWSKI,wyplata
7,2016-11-10,59007,MICHAL,-2500.00,WALKOWSKI,wyplata
8,2016-12-11,59008,MICHAL,-2500.00,WALKOWSKI,wyplata
9,2017-01-10,59009,MICHAL,-2500.00,WALKOWSKI,wyplata


In [25]:
pd.DataFrame(hist_zdr)

,ID pracownika,ID_wizyty,data wizyty,diagnoza,id,stan zdrowia,sterylizacja,zalecenia
0,30002,43000,2018-11-27,skaleczenie,61000,zdrowy,tak,lek
1,30001,42001,2019-03-19,inne,61001,zdrowy,tak,lek
2,30003,42002,2018-03-31,świeżb,61002,zdrowy,tak,lek
3,30000,43003,2019-05-04,przeziębienie,61003,zdrowy,tak,lek
4,30001,42004,2018-11-20,skaleczenie,61004,chory,tak,lek
5,30003,41005,2019-04-04,inne,61005,zdrowy,nie,lek
6,30001,43006,2019-03-06,przeziębienie,61006,chory,tak,lek
7,30001,42007,2019-06-18,inne,61007,w czasie leczenia,tak,lek
8,30003,41008,2019-03-15,inne,61008,zdrowy,tak,lek
9,30000,42009,2018-03-10,świeżb,61009,zdrowy,tak,lek


In [26]:
pd.DataFrame(zwierz_schr)

,ID opiekuna,Stan,blok,id,id_gat
0,[20001],nadaje się do adopcji,B_1,61000,1
1,[20000],nadaje się do adopcji,A_1,61001,1
2,[20001],nadaje się do adopcji,B_2,61002,1
3,[20001],nadaje się do adopcji,B_3,61003,1
4,[20000],nie nadaje się do adopcji - chory,A_2,61004,1
5,[20000],nadaje się do adopcji,A_3,61005,1
6,[20000],nie nadaje się do adopcji - chory,A_4,61006,1
7,[20001],nie nadaje się do adopcji - w czasie leczenia,B_4,61007,1
8,[20001],nadaje się do adopcji,B_5,61008,1
9,[20001],nadaje się do adopcji,B_6,61009,1


In [27]:
pd.DataFrame(informacje)

,ID opiekuna,blok,data_przyjecia,data_urodzenia,gatunek,id,id_gat,imie,inne_zwierzeta,masa,plec,umaszczenie,wcześniej adoptowany
0,[20001],B_1,2018-09-30,2005-12-14,pies,61000,1,Nicpon,łagodny,19.366,samiec,szary,tak
1,[20000],A_1,2016-07-10,2012-01-09,pies,61001,1,Walter,łagodny,22.434,samiec,rudy,nie
2,[20001],B_2,2017-02-22,2013-03-23,pies,61002,1,Demi,łagodny,15.170,samica,bialo-szary,nie
3,[20001],B_3,2019-03-05,2006-07-31,pies,61003,1,Nira,łagodny,17.479,samica,szary,tak
4,[20000],A_2,2018-01-07,2007-02-05,pies,61004,1,Ora,agresywny,24.996,samica,inne,nie
5,[20000],A_3,2016-08-30,2007-12-12,pies,61005,1,Kari,agresywny,22.662,samica,bialo-rudy,nie
6,[20000],A_4,2018-10-14,2007-12-02,pies,61006,1,Nora,łagodny,25.632,samica,inne,nie
7,[20001],B_4,2018-05-07,2013-04-30,pies,61007,1,Uchacz,łagodny,16.319,samiec,inne,nie
8,[20001],B_5,2019-01-18,2013-05-10,pies,61008,1,Fik,łagodny,14.959,samiec,czarny,nie
9,[20001],B_6,2016-11-14,2014-01-25,pies,61009,1,Naomi,łagodny,17.178,samica,bialo-brązowy,nie


## Zaadoptowane

### pieski

In [28]:
#92 psy- tyle liczy aktualnie nasze schronisko, 41 kotow, i 17 szczurow
len_dog = 92
len_ad = 139
k = 1
l = 1
for i in range(len_dog, len_dog+len_ad):
    dog_info = {}
    dog_byl = {}
    dog_wiz = {}
    dog_trans = {}
    dog_ad = {}
    dog = Animal(species = 'pies')
    birth, _ = dog.birth()
    birth_d = dataformat(birth)
    acc = dog.acceptance()
    if np.random.random()>0.7:
        dog_info['wcześniej adoptowany'] = "tak"
        adop = dog.adoption()
        #ponowne przyjecie
        acc = dog.next_acceptance()
        if np.random.random()>0.9:
            adop = dog.adoption()
            #ponowne przyjecie
            acc = dog.next_acceptance()
    else:
        dog_info['wcześniej adoptowany'] = "nie"
        
    adoption = dog.adoption()
    ad_date = dataformat(adoption)
    #adoptujacy
    rand_i = int(np.random.uniform(0,2.49))
    if np.random.random()>0.5:
        adoptujacy = dog.adoptujacy(names_male, lastnames_male, ulice, miasta[rand_i])
    else:
        adoptujacy = dog.adoptujacy(names_female, lastnames_female, ulice, miasta[rand_i])

    dog_ad['id adoptujacego'] = id_generator(i=i, first=7, second =1)
    dog_ad['imie'] = adoptujacy['name']
    dog_ad['nazwisko'] = adoptujacy['last_name']
    dog_ad['telefon'] = adoptujacy['telefon']
    dog_ad['adres'] = adoptujacy['adres']
    dog_ad['ilosc adopcji'] = 1
    dog_info['data_urodzenia'] = birth_d
    dog_info['data_przyjecia'] = dataformat(acc)
    
    dog_info['plec'] = dog.plec()
    dog_info['masa'] = dog.masa()
    dog_info['imie'] = dog.name('./dane/npieski.csv')
    kastracja = dog.kastracja()
    dog_info['id_gat'] = 1
    dog_schr['id_gat'] = dog_info['id_gat']
    dog_info['id'] = id_generator(i=i, first=6, second =1)
        
    dog_byl['id'] = dog_info['id']
    dog_byl['id wydania'] = id_generator(i=i, first=8, second =1)
    dog_byl['przyczyna'] = 'adopcja'
    dog_byl['data'] = ad_date
    dog_byl['id adoptujacego'] = dog_ad['id adoptujacego']
        
    if np.random.random()>0.9:
        dog_info['inne_zwierzeta'] = "agresywny"
    else:
        dog_info['inne_zwierzeta'] = "łagodny"
    umaszczenie= dog.umaszczenie()
    
    dog_info['umaszczenie']=umaszczenie[0]
    dog_info['gatunek'] = umaszczenie[1]
    
    if dog_info['masa']>=20:
        dog_info['ID opiekuna'] = [pr_id[0]]
        dog_info['blok'] = 'A_'+str(k)
        k += 1
    else:
        dog_info['ID opiekuna'] = [pr_id[1]]
        dog_info['blok'] = 'B_'+str(l)
        l += 1

    ## czy nadaje sie do adopcji?
    if np.random.random()>0.1:
        data_wiz, chor, zalec, cena = dog.Wizyty()
        if np.random.random()>0.7:
            data_wiz1, chor1, zalec1, cena1 = dog.Wizyty(ad_time = adoption)
    zdrowie = dog.stan_zdrowia()
    if zdrowie  == "zdrowy":
        dog_schr['Stan'] = "nadaje się do adopcji"
    else:
        dog_schr['Stan'] = "nie nadaje się do adopcji - "+str(zdrowie)
    wizyty, daty, ceny = dog.choroby()
    for j in range(len(wizyty)):
        dog_wiz['ID_wizyty'] =  id_generator(i = i, first = 4, second = j)
        dog_wiz['id'] = dog_info['id']
        index =int(np.round(np.random.uniform(0,3.49),0))
        id_pr =  w_id[index]
        dog_wiz['ID pracownika'] = id_pr
        dog_wiz['data wizyty'] = dataformat(daty[j])
        dog_wiz['diagnoza'] = wizyty[j]
        dog_wiz['zalecenia'] = 'lek'
        dog_wiz['stan zdrowia'] = zdrowie
        dog_wiz['sterylizacja'] = kastracja[0]
        dog_trans['id_transakcji'] = id_generator(i = i, first = 5, second = j)
        dog_trans['tytul']='weterynarz, id_zwierzecia:'+ str(dog_info['id'])
        dog_trans['kwota'] = 0 - ceny[j]
        dog_trans['data'] = dataformat(daty[j])
        dog_trans['imie'] = weterynarze[index]['imie']
        dog_trans['nazwisko'] = weterynarze[index]['nazwisko']
    transakcje.append(dog_trans)
    hist_zdr.append(dog_wiz)
    zwierz_schr.append(dog_schr)
    informacje.append(dog_info)
    adoptujace.append(dog_ad)
    zwierz_byle.append(dog_byl)
    
    

In [29]:
#92 psy- tyle liczy aktualnie nasze schronisko, 41 kotow, i 17 szczurow
len_dog = 41
len_ad = 96
for i in range(len_dog, len_dog+len_ad):
    dog_info = {}
    dog_byl = {}
    dog_wiz = {}
    dog_trans = {}
    dog_ad = {}
    dog = Animal(species = 'kot')
    birth, _ = dog.birth()
    birth_d = dataformat(birth)
    acc = dog.acceptance()
    if np.random.random()>0.7:
        dog_info['wcześniej adoptowany'] = "tak"
        adop = dog.adoption()
        #ponowne przyjecie
        acc = dog.next_acceptance()
        if np.random.random()>0.9:
            adop = dog.adoption()
            #ponowne przyjecie
            acc = dog.next_acceptance()
    else:
        dog_info['wcześniej adoptowany'] = "nie"
        
    adoption = dog.adoption()
    ad_date = dataformat(adoption)
    #adoptujacy
    rand_i = int(np.random.uniform(0,2.49))
    if np.random.random()>0.5:
        adoptujacy = dog.adoptujacy(names_male, lastnames_male, ulice, miasta[rand_i])
    else:
        adoptujacy = dog.adoptujacy(names_female, lastnames_female, ulice, miasta[rand_i])

    dog_ad['id adoptujacego'] = id_generator(i=i, first=7, second =1)
    dog_ad['imie'] = adoptujacy['name']
    dog_ad['nazwisko'] = adoptujacy['last_name']
    dog_ad['telefon'] = adoptujacy['telefon']
    dog_ad['adres'] = adoptujacy['adres']
    dog_ad['ilosc adopcji'] = 1
    dog_info['data_urodzenia'] = birth_d
    dog_info['data_przyjecia'] = dataformat(acc)
    
    dog_info['plec'] = dog.plec()
    dog_info['masa'] = dog.masa()
    dog_info['imie'] = dog.name('./dane/npieski.csv')
    kastracja = dog.kastracja()
    dog_info['id_gat'] = 2
    dog_schr['id_gat'] = dog_info['id_gat']
    dog_info['id'] = id_generator(i=i, first=6, second =2)
        
    dog_byl['id'] = dog_info['id']
    dog_byl['id wydania'] = id_generator(i=i, first=8, second =2)
    dog_byl['przyczyna'] = 'adopcja'
    dog_byl['data'] = ad_date
    dog_byl['id adoptujacego'] = dog_ad['id adoptujacego']
        
    if np.random.random()>0.9:
        dog_info['inne_zwierzeta'] = "agresywny"
    else:
        dog_info['inne_zwierzeta'] = "łagodny"
    umaszczenie= dog.umaszczenie()
    
    dog_info['umaszczenie']=umaszczenie[0]
    dog_info['gatunek'] = umaszczenie[1]
    
    dog_info['ID opiekuna'] = [pr_id[3]]
    if i%3==0:
        dog_info['blok'] = 'C_'+str(i//3)
        k = i
    else:
        dog_info['blok'] = 'C_'+str(k//3)
        
    ## czy nadaje sie do adopcji?
    if np.random.random()>0.1:
        data_wiz, chor, zalec, cena = dog.Wizyty()
        if np.random.random()>0.7:
            data_wiz1, chor1, zalec1, cena1 = dog.Wizyty(ad_time = adoption)
    zdrowie = dog.stan_zdrowia()
    if zdrowie  == "zdrowy":
        dog_schr['Stan'] = "nadaje się do adopcji"
    else:
        dog_schr['Stan'] = "nie nadaje się do adopcji - "+str(zdrowie)
    wizyty, daty, ceny = dog.choroby()
    for j in range(len(wizyty)):
        dog_wiz['ID_wizyty'] =  id_generator(i = i, first = 4, second = j)
        dog_wiz['id'] = dog_info['id']
        index =int(np.round(np.random.uniform(0,3.49),0))
        id_pr =  w_id[index]
        dog_wiz['ID pracownika'] = id_pr
        dog_wiz['data wizyty'] = dataformat(daty[j])
        dog_wiz['diagnoza'] = wizyty[j]
        dog_wiz['zalecenia'] = 'lek'
        dog_wiz['stan zdrowia'] = zdrowie
        dog_wiz['sterylizacja'] = kastracja[0]
        dog_trans['id_transakcji'] = id_generator(i = i, first = 5, second = j)
        dog_trans['tytul']='weterynarz, id_zwierzecia:'+ str(dog_info['id'])
        dog_trans['kwota'] = 0-ceny[j]
        dog_trans['data'] = dataformat(daty[j])
        dog_trans['imie'] = weterynarze[index]['imie']
        dog_trans['nazwisko'] = weterynarze[index]['nazwisko']
    transakcje.append(dog_trans)
    hist_zdr.append(dog_wiz)
    zwierz_schr.append(dog_schr)
    informacje.append(dog_info)
    adoptujace.append(dog_ad)
    zwierz_byle.append(dog_byl)
    
    

## Zmarle zwierzeta


In [30]:
#17 psow i 6 kotow zmarlo w schornisku
#92 psy- tyle liczy aktualnie nasze schronisko, 41 kotow, i 17 szczurow
len_dog = 92
len_ad = 139
len_d = 17
k = 1
l = 1
for i in range(len_dog+len_ad, len_dog+len_ad+len_d):
    dog_info = {}
    dog_byl = {}
    dog_wiz = {}
    dog_trans = {}
    dog_ad = {}
    dog = Animal(species = 'pies')
    death_time = np.random.uniform(0.01, 3)
    birth, death = dog.birth(death_year = death_time)
    birth_d = dataformat(birth)
    acc = dog.acceptance()
    if np.random.random()>0.7:
        dog_info['wcześniej adoptowany'] = "tak"
        adop = dog.adoption()
        #ponowne przyjecie
        acc = dog.next_acceptance()
        if np.random.random()>0.9:
            adop = dog.adoption()
            #ponowne przyjecie
            acc = dog.next_acceptance()
    else:
        dog_info['wcześniej adoptowany'] = "nie"
        
    dog_info['data_urodzenia'] = birth_d
    dog_info['data_przyjecia'] = dataformat(acc)
    
    dog_info['plec'] = dog.plec()
    dog_info['masa'] = dog.masa()
    dog_info['imie'] = dog.name('./dane/npieski.csv')
    kastracja = dog.kastracja()
    dog_info['id_gat'] = 1
    dog_schr['id_gat'] = dog_info['id_gat']
    dog_info['id'] = id_generator(i=i, first=6, second =1)
        
    dog_byl['id'] = dog_info['id']
    dog_byl['id wydania'] = 111111 #jesli same 1 to oznacza smierc
    dog_byl['przyczyna'] = 'zgon'
    dog_byl['data'] = dataformat(death)
    dog_byl['id adoptujacego'] = 'brak'
        
    if np.random.random()>0.9:
        dog_info['inne_zwierzeta'] = "agresywny"
    else:
        dog_info['inne_zwierzeta'] = "łagodny"
    umaszczenie= dog.umaszczenie()
    
    dog_info['umaszczenie']=umaszczenie[0]
    dog_info['gatunek'] = umaszczenie[1]
    
    if dog_info['masa']>=20:
        dog_info['ID opiekuna'] = [pr_id[0]]
        dog_info['blok'] = 'A_'+str(k)
        k += 1
    else:
        dog_info['ID opiekuna'] = [pr_id[1]]
        dog_info['blok'] = 'B_'+str(l)
        l += 1

    ## czy nadaje sie do adopcji?
    if np.random.random()>0.1:
        data_wiz, chor, zalec, cena = dog.Wizyty()
        if np.random.random()>0.7:
            data_wiz1, chor1, zalec1, cena1 = dog.Wizyty(ad_time = adoption)
    zdrowie = dog.stan_zdrowia()
    if zdrowie  == "zdrowy":
        dog_schr['Stan'] = "nadaje się do adopcji"
    else:
        dog_schr['Stan'] = "nie nadaje się do adopcji - "+str(zdrowie)
    wizyty, daty, ceny = dog.choroby()
    for j in range(len(wizyty)):
        dog_wiz['ID_wizyty'] =  id_generator(i = i, first = 4, second = j)
        dog_wiz['id'] = dog_info['id']
        index =int(np.round(np.random.uniform(0,3.49),0))
        id_pr =  w_id[index]
        dog_wiz['ID pracownika'] = id_pr
        dog_wiz['data wizyty'] = dataformat(daty[j])
        dog_wiz['diagnoza'] = wizyty[j]
        dog_wiz['zalecenia'] = 'lek'
        dog_wiz['stan zdrowia'] = zdrowie
        dog_wiz['sterylizacja'] = kastracja[0]
        dog_trans['id_transakcji'] = id_generator(i = i, first = 5, second = j)
        dog_trans['tytul']='weterynarz, id_zwierzecia:'+ str(dog_info['id'])
        dog_trans['kwota'] = 0-ceny[j]
        dog_trans['data'] = dataformat(daty[j])
        dog_trans['imie'] = weterynarze[index]['imie']
        dog_trans['nazwisko'] = weterynarze[index]['nazwisko']
    transakcje.append(dog_trans)
    hist_zdr.append(dog_wiz)
    zwierz_schr.append(dog_schr)
    informacje.append(dog_info)
    adoptujace.append(dog_ad)
    zwierz_byle.append(dog_byl)
    
    

In [31]:
#17 psow i 6 kotow zmarlo w schornisku
#92 psy- tyle liczy aktualnie nasze schronisko, 41 kotow, i 17 szczurow
len_dog = 41
len_ad = 96
len_d = 6
k = 1
l = 1
for i in range(len_dog+len_ad, len_dog+len_ad+len_d):
    dog_info = {}
    dog_byl = {}
    dog_wiz = {}
    dog_trans = {}
    dog_ad = {}
    dog = Animal(species = 'kot')
    death_time = np.random.uniform(0.01, 3)
    birth, death = dog.birth(death_year = death_time)
    birth_d = dataformat(birth)
    acc = dog.acceptance()
    if np.random.random()>0.7:
        dog_info['wcześniej adoptowany'] = "tak"
        adop = dog.adoption()
        #ponowne przyjecie
        acc = dog.next_acceptance()
        if np.random.random()>0.9:
            adop = dog.adoption()
            #ponowne przyjecie
            acc = dog.next_acceptance()
    else:
        dog_info['wcześniej adoptowany'] = "nie"
        
    dog_info['data_urodzenia'] = birth_d
    dog_info['data_przyjecia'] = dataformat(acc)
    
    dog_info['plec'] = dog.plec()
    dog_info['masa'] = dog.masa()
    dog_info['imie'] = dog.name('./dane/npieski.csv')
    kastracja = dog.kastracja()
    dog_info['id_gat'] = 1
    dog_schr['id_gat'] = dog_info['id_gat']
    dog_info['id'] = id_generator(i=i, first=6, second =2)
        
    dog_byl['id'] = dog_info['id']
    dog_byl['id wydania'] = 111111 #jesli same 1 to oznacza smierc
    dog_byl['przyczyna'] = 'zgon'
    dog_byl['data'] = dataformat(death)
    dog_byl['id adoptujacego'] = 'brak'
        
    if np.random.random()>0.9:
        dog_info['inne_zwierzeta'] = "agresywny"
    else:
        dog_info['inne_zwierzeta'] = "łagodny"
    umaszczenie= dog.umaszczenie()
    
    dog_info['umaszczenie']=umaszczenie[0]
    dog_info['gatunek'] = umaszczenie[1]
    
    dog_info['ID opiekuna'] = [pr_id[3]]
    if i%3==0:
        dog_info['blok'] = 'C_'+str(i//3)
        k = i
    else:
        dog_info['blok'] = 'C_'+str(k//3)
        
    ## czy nadaje sie do adopcji?
    if np.random.random()>0.1:
        data_wiz, chor, zalec, cena = dog.Wizyty()
        if np.random.random()>0.7:
            data_wiz1, chor1, zalec1, cena1 = dog.Wizyty(ad_time = adoption)
    zdrowie = dog.stan_zdrowia()
    if zdrowie  == "zdrowy":
        dog_schr['Stan'] = "nadaje się do adopcji"
    else:
        dog_schr['Stan'] = "nie nadaje się do adopcji - "+str(zdrowie)
    wizyty, daty, ceny = dog.choroby()
    for j in range(len(wizyty)):
        dog_wiz['ID_wizyty'] =  id_generator(i = i, first = 4, second = j)
        dog_wiz['id'] = dog_info['id']
        index =int(np.round(np.random.uniform(0,3.49),0))
        id_pr =  w_id[index]
        dog_wiz['ID pracownika'] = id_pr
        dog_wiz['data wizyty'] = dataformat(daty[j])
        dog_wiz['diagnoza'] = wizyty[j]
        dog_wiz['zalecenia'] = 'lek'
        dog_wiz['stan zdrowia'] = zdrowie
        dog_wiz['sterylizacja'] = kastracja[0]
        dog_trans['id_transakcji'] = id_generator(i = i, first = 5, second = j)
        dog_trans['tytul']='weterynarz, id_zwierzecia:'+ str(dog_info['id'])
        dog_trans['kwota'] = 0-ceny[j]
        dog_trans['data'] = dataformat(daty[j])
        dog_trans['imie'] = weterynarze[index]['imie']
        dog_trans['nazwisko'] = weterynarze[index]['nazwisko']
    transakcje.append(dog_trans)
    hist_zdr.append(dog_wiz)
    zwierz_schr.append(dog_schr)
    informacje.append(dog_info)
    adoptujace.append(dog_ad)
    zwierz_byle.append(dog_byl)
    
    

### Darowizny i dotacje

In [32]:
#comiesieczna dotacja z urzedu - 40 000 
data, pensja = add_salary(open_time,actual_time,40000)
for j in range(len(data)):
    trans = {}
    trans['data'] = data[j]
    trans['id_transakcji'] = id_generator(i=j, first=5, second = 9)
    trans['imie'] = opiekun['name']
    trans['nazwisko'] = opiekun['last_name']
    trans['kwota'] = float(pensja[j])
    trans['tytul'] = 'wyplata'
    transakcje.append(trans)

In [33]:
#darowizny - losowe
#ilosc darowizn w ciagu trzech lat- zalozmy ze ok 1500-2000
amount =int(np.random.uniform(150, 900))
for i in range(amount):
    if np.random.random()>0.1:
        kwota = abs(np.round(np.random.normal(50, 100),2))
    else:
        kwota = np.round(np.random.uniform(1,30000),2)
    rand_i = int(np.random.uniform(0,2.49))
    if rand>0.5:
        opiekun = pracownik(names_male, lastnames_male, ulice, miasta[rand_i])
    else:
        opiekun = pracownik(names_female, lastnames_female, ulice, miasta[rand_i])
    trans = {}
    data = dataformat(np.random.uniform(open_time, actual_time))
    trans['data'] = data
    trans['id_transakcji'] = id_generator(i=i, first=5, second = 7)
    trans['imie'] = opiekun['name']
    trans['nazwisko'] = opiekun['last_name']
    trans['kwota'] = kwota
    trans['tytul'] = 'Darowizna'
    transakcje.append(trans)

In [34]:
pd.DataFrame(transakcje)

,data,id_transakcji,imie,kwota,nazwisko,tytul
0,2016-04-10,59000,MICHAL,-532.83,WALKOWSKI,wyplata
1,2016-05-11,59001,MICHAL,-2500.00,WALKOWSKI,wyplata
2,2016-06-11,59002,MICHAL,-2500.00,WALKOWSKI,wyplata
3,2016-07-11,59003,MICHAL,-2500.00,WALKOWSKI,wyplata
4,2016-08-11,59004,MICHAL,-2500.00,WALKOWSKI,wyplata
5,2016-09-10,59005,MICHAL,-2500.00,WALKOWSKI,wyplata
6,2016-10-11,59006,MICHAL,-2500.00,WALKOWSKI,wyplata
7,2016-11-10,59007,MICHAL,-2500.00,WALKOWSKI,wyplata
8,2016-12-11,59008,MICHAL,-2500.00,WALKOWSKI,wyplata
9,2017-01-10,59009,MICHAL,-2500.00,WALKOWSKI,wyplata


### Zapisywanie do csv

In [35]:
df_info = pd.DataFrame(informacje)
df_info.to_csv(r'.\dane\informacje.csv')
df_gat = pd.DataFrame(gatunki)
df_gat.to_csv(r'.\dane\gatunki.csv')
df_schr = pd.DataFrame(zwierz_schr)
df_schr.to_csv(r'.\dane\zwierz_schr.csv')
df_byle = pd.DataFrame(zwierz_byle)
df_byle.to_csv(r'.\dane\zwierz_byle.csv')
df_adop = pd.DataFrame(adoptujace)
df_adop.to_csv(r'.\dane\adoptujace.csv')
df_hist = pd.DataFrame(hist_zdr)
df_hist.to_csv(r'.\dane\hist_zdr.csv')
df_trans = pd.DataFrame(transakcje)
df_trans.to_csv(r'.\dane\transakcje.csv')

In [36]:
df_info

,ID opiekuna,blok,data_przyjecia,data_urodzenia,gatunek,id,id_gat,imie,inne_zwierzeta,masa,plec,umaszczenie,wcześniej adoptowany
0,[20001],B_1,2018-09-30,2005-12-14,pies,61000,1,Nicpon,łagodny,19.366,samiec,szary,tak
1,[20000],A_1,2016-07-10,2012-01-09,pies,61001,1,Walter,łagodny,22.434,samiec,rudy,nie
2,[20001],B_2,2017-02-22,2013-03-23,pies,61002,1,Demi,łagodny,15.170,samica,bialo-szary,nie
3,[20001],B_3,2019-03-05,2006-07-31,pies,61003,1,Nira,łagodny,17.479,samica,szary,tak
4,[20000],A_2,2018-01-07,2007-02-05,pies,61004,1,Ora,agresywny,24.996,samica,inne,nie
5,[20000],A_3,2016-08-30,2007-12-12,pies,61005,1,Kari,agresywny,22.662,samica,bialo-rudy,nie
6,[20000],A_4,2018-10-14,2007-12-02,pies,61006,1,Nora,łagodny,25.632,samica,inne,nie
7,[20001],B_4,2018-05-07,2013-04-30,pies,61007,1,Uchacz,łagodny,16.319,samiec,inne,nie
8,[20001],B_5,2019-01-18,2013-05-10,pies,61008,1,Fik,łagodny,14.959,samiec,czarny,nie
9,[20001],B_6,2016-11-14,2014-01-25,pies,61009,1,Naomi,łagodny,17.178,samica,bialo-brązowy,nie


In [37]:
df_gat

,nazwa,id_gatunku
0,pies,1
1,kot,2
2,szczur,3


In [38]:
df_schr

,ID opiekuna,Stan,blok,id,id_gat
0,[20001],nadaje się do adopcji,B_1,61000,1
1,[20000],nadaje się do adopcji,A_1,61001,1
2,[20001],nadaje się do adopcji,B_2,61002,1
3,[20001],nadaje się do adopcji,B_3,61003,1
4,[20000],nie nadaje się do adopcji - chory,A_2,61004,1
5,[20000],nadaje się do adopcji,A_3,61005,1
6,[20000],nie nadaje się do adopcji - chory,A_4,61006,1
7,[20001],nie nadaje się do adopcji - w czasie leczenia,B_4,61007,1
8,[20001],nadaje się do adopcji,B_5,61008,1
9,[20001],nadaje się do adopcji,B_6,61009,1


In [39]:
df_byle

,data,id,id adoptujacego,id wydania,przyczyna
0,2018-12-27,61092,71092,81092,adopcja
1,2019-05-14,61093,71093,81093,adopcja
2,2019-04-18,61094,71094,81094,adopcja
3,2018-07-29,61095,71095,81095,adopcja
4,2019-06-11,61096,71096,81096,adopcja
5,2019-05-14,61097,71097,81097,adopcja
6,2019-04-03,61098,71098,81098,adopcja
7,2019-05-04,61099,71099,81099,adopcja
8,2019-03-05,61100,71100,81100,adopcja
9,2019-01-14,61101,71101,81101,adopcja


In [40]:
df_adop

,adres,id adoptujacego,ilosc adopcji,imie,nazwisko,telefon
0,ul. Spacerowa 66 Olesnica,71092,1.0,SZYMON,RUSZEL,331-781-557
1,ul. Jastrzebia 16 Wroclaw,71093,1.0,RENATA,CHABROS,722-102-270
2,ul. Henryka Lasaka 82 Opole,71094,1.0,PAWEL,MALEC,122-277-420
3,ul. 11 Listopada 20 Wroclaw,71095,1.0,GRZEGORZ,CHOJNOWSKI,496-167-475
4,ul. sw. Jadwigi 40 Olesnica,71096,1.0,JADWIGA,MALISZEWSKA,896-266-246
5,ul. Jana Stapinskiego 32 Wroclaw,71097,1.0,ROBERT,KULIGOWSKI,574-459-994
6,ul. Leszczynowa 85 Opole,71098,1.0,ROBERT,LASECKI,793-291-671
7,ul. Bronislawa Malinowskiego 74 Olesnica,71099,1.0,DAMIAN,MROWIEC,855-172-874
8,ul. sw. Jadwigi 90 Wroclaw,71100,1.0,RYSZARD,SOKULSKI,826-925-866
9,ul. Batalionow Chlopskich 18 Opole,71101,1.0,RAFAL,KONIECZKO,341-434-635


In [41]:
df_hist

,ID pracownika,ID_wizyty,data wizyty,diagnoza,id,stan zdrowia,sterylizacja,zalecenia
0,30002,43000,2018-11-27,skaleczenie,61000,zdrowy,tak,lek
1,30001,42001,2019-03-19,inne,61001,zdrowy,tak,lek
2,30003,42002,2018-03-31,świeżb,61002,zdrowy,tak,lek
3,30000,43003,2019-05-04,przeziębienie,61003,zdrowy,tak,lek
4,30001,42004,2018-11-20,skaleczenie,61004,chory,tak,lek
5,30003,41005,2019-04-04,inne,61005,zdrowy,nie,lek
6,30001,43006,2019-03-06,przeziębienie,61006,chory,tak,lek
7,30001,42007,2019-06-18,inne,61007,w czasie leczenia,tak,lek
8,30003,41008,2019-03-15,inne,61008,zdrowy,tak,lek
9,30000,42009,2018-03-10,świeżb,61009,zdrowy,tak,lek


In [42]:
df_trans

,data,id_transakcji,imie,kwota,nazwisko,tytul
0,2016-04-10,59000,MICHAL,-532.83,WALKOWSKI,wyplata
1,2016-05-11,59001,MICHAL,-2500.00,WALKOWSKI,wyplata
2,2016-06-11,59002,MICHAL,-2500.00,WALKOWSKI,wyplata
3,2016-07-11,59003,MICHAL,-2500.00,WALKOWSKI,wyplata
4,2016-08-11,59004,MICHAL,-2500.00,WALKOWSKI,wyplata
5,2016-09-10,59005,MICHAL,-2500.00,WALKOWSKI,wyplata
6,2016-10-11,59006,MICHAL,-2500.00,WALKOWSKI,wyplata
7,2016-11-10,59007,MICHAL,-2500.00,WALKOWSKI,wyplata
8,2016-12-11,59008,MICHAL,-2500.00,WALKOWSKI,wyplata
9,2017-01-10,59009,MICHAL,-2500.00,WALKOWSKI,wyplata
